In [1]:
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import pickle

### Comienzo de la appy 

In [200]:
# df = pd.read_csv("Data\codigos_postales_municipios.csv",  dtype = {"codigo_postal" : str} )

In [201]:
# codigo_postal = df["codigo_postal"][10957:14609].to_list()

# i = 0

# diccionario = {}

# for codigo in tqdm(codigo_postal):
#     i = i + 1

#     url = "https://api.openchargemap.io/v3/poi/"
#     parametros = {
#                     "output": "json",
#                     "countrycode": "ES",
#                     "maxresults": 500,
#                     "postalcode": codigo,
#                     "key": "7aff135a-a726-4a4d-9b06-dd242186c9f1"  # Tu clave aquí
#                 }

#     try:
#         respuesta = requests.get(url, params=parametros)
#         respuesta.raise_for_status()  # Lanza una excepción si la solicitud falla
#         # print(f"Quedan {len(codigo_postal) - i} codigos postales")

#         datos = respuesta.json()

#         diccionario[codigo] = datos


#     except requests.exceptions.RequestException as e:
#         print("Error al hacer la solicitud:", e)
# resul = pd.DataFrame(diccionario)

In [202]:
# resul.shape

In [203]:
# resul.to_csv("Dani.csv")

### Fin de la appy

### -----------------------------------------------EDA-----------------------------------------------------------###

In [2]:
df = pd.read_csv("Data\df_coches_escrapeo.csv")

In [3]:
# copia del df original a fin de realizar comparativas
df_coches = df.copy()

#Limpieza del df para quitar los camiones autobuses y otros vehiculos que no sean coches o furgonetas
df_coches = df_coches[(df_coches["MTMA (Kg)"] > 400) & (df_coches["MTMA (Kg)"] < 3000)] 
df_coches = df_coches[df_coches["Motorización"].notnull()]
df_coches = df_coches[~(df_coches["Motorización"] == "Pila de combustible")]
df_coches = df_coches[~(df_coches["Motorización"] == "Autonomía extendida")]
#Fin de la limpieza de los coches que no son turismos o furgonetas

# Unir los que son de gases licuados y gas natural
df_coches.loc[df_coches["Motorización"] == "Gases licuados del petróleo (GLP)", "Motorización"] = "Gas natural"
#Corregir el consumo del coche que venia erroneo
df_coches.loc[(df_coches['Motorización'] == 'Gasolina') & (df_coches['Consumo Mínimo (l/100km)'] == 60), 'Consumo Mínimo (l/100km)'] = 6

 
#quitar los electricos puros que tienen consumo de combustion
df_coches.loc[df_coches['Motorización'] == 'Eléctricos puros', ['Consumo Mínimo (l/100km)', 'Consumo Máximo (l/100km)']] = 0
df_coches.reset_index(inplace = True , drop = True)

In [11]:
type(df_coches[df_coches['Modelo'] == 'AIWAYS U5 MAS861-WVTA/2WB/FL4'])

pandas.core.frame.DataFrame

In [6]:
df_coches['Modelo'].values[0]

'AIWAYS U5 MAS861-WVTA/2WB/FL4'

In [7]:
df_coches['Modelo'].values[0].split(" ")[0]

'AIWAYS'

In [12]:
def funcion_line_precio(combustible):

    df = pd.read_csv("../data/Precios_Gasolina_y_Precios_Gasoleo.csv")
    df["Semana"] = df["Semana"].apply(lambda x: x.split(" ")[1])
    df['Fecha'] = df.apply(lambda row: pd.to_datetime(f'{int(row["Año"])}-W{int(row["Semana"])}-1', format='%G-W%V-%u'), axis=1)
    fig = px.line(data_frame = df, 
                  x = "Fecha", 
                  y = f"Precio {combustible.capitalize()}")
    return fig

In [14]:

def funcion_line_precio():
    df = pd.read_csv(r"D:\bootcamp\Proyecto3\proyecto3sucio\Precios_Gasolina_y_Precios_Gasoleo.csv")
    df["Semana"] = df["Semana"].apply(lambda x: x.split(" ")[1])
    df['Fecha'] = df.apply(lambda row: pd.to_datetime(f'{int(row["Año"])}-W{int(row["Semana"])}-1', format='%G-W%V-%u'), axis=1)

    fig = px.line(data_frame=df, 
                  x="Fecha", 
                  y=["Precio Gasolina", "Precio Gasoleo"],
                  labels={"value": "Precio", "variable": "Tipo de Combustible"},
                  color_discrete_map={"Precio Gasolina": "blue", "Precio Gasoleo": "green"})

    return fig

# Ejemplo de uso
figura_combustibles = funcion_line_precio()
figura_combustibles.show()

In [4]:
#Comprobacion de las filas que se han eliminado
df.shape,df_coches.shape

((16322, 33), (13365, 33))

In [5]:
df_coches[df_coches['Motorización'] == 'Gasolina']

,Modelo,Consumo Mínimo (l/100km),Consumo Máximo (l/100km),Emisiones Mínimo,Emisiones Máximo,MTMA (Kg),Motorización,Consumo Eléctrico (kWh/100km),Potencia eléctrica (kW),Autonomía eléctrica (km),...,N2,N3,A,B,C,D,E,F,G,Sin clasificacion
23,Abarth 595 ABARTH 595 1.4 16v T-JET 107 KW (14...,7.2,7.4,162.0,165.0,1045.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
24,Abarth 595 ABARTH 595 COMPETIZIONE 1.4 16v T-J...,7.7,7.8,171.0,172.0,1065.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
25,Abarth 595 ABARTH 595 ESSEESSE 1.4 16v T-JET 1...,7.7,7.8,171.0,172.0,1065.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
26,Abarth 595 ABARTH 595 MONSTER ENERGY YAMAHA 1....,7.4,7.5,165.0,168.0,1065.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
27,Abarth 595 ABARTH 595 PISTA 1.4 16v T-JET 121 ...,7.4,7.5,165.0,168.0,1065.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13200,"Volvo XC40 Gasolina, Essential, Core, Plus, Au...",7.2,7.9,162.0,179.0,2120.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
13206,"Volvo XC40 Momentum Core, Momentum Pro, Inscri...",7.2,7.9,162.0,179.0,2120.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
13207,"Volvo XC40 Momentum Core, Momentum Pro, Inscri...",6.8,7.6,154.0,173.0,2080.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
13208,"Volvo XC40 Momentum Core, Momentum Pro, Inscri...",7.2,7.9,162.0,179.0,2120.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### Calculo del precio de la gasolina por km

In [67]:
df_coches.Motorización.unique()

array(['Eléctricos puros', 'Gasolina', 'Gasóleo', 'Híbridos enchufables',
       'Híbridos de gasolina', 'Gas natural', 'Híbridos de gasóleo',
       'Gases licuados del petróleo (GLP)'], dtype=object)

In [15]:
#df_coches[(df_coches['Motorización'] == 'Gasolina') | (df_coches['Motorización'] == 'Gasóleo')][['Consumo Mínimo (l/100km)','Consumo Máximo (l/100km)']]

df_coches["Consumo medio l/1km"] = df_coches[["Consumo Máximo (l/100km)", "Consumo Mínimo (l/100km)"]].mean(axis=1)/100

In [16]:
df_coches[df_coches['Motorización'] == 'Gasolina'][['Modelo']].values

array([['Abarth 595 ABARTH 595 1.4 16v T-JET 107 KW (145 CV) E6D'],
       ['Abarth 595 ABARTH 595 COMPETIZIONE 1.4 16v T-JET 132 KW (180 CV) E6D'],
       ['Abarth 595 ABARTH 595 ESSEESSE 1.4 16v T-JET 132 KW (180 CV) E6D'],
       ...,
       ['Volvo XC40 Momentum Core, Momentum Pro, Inscription, R-design, T2, Manual, MY22'],
       ['Volvo XC40 Momentum Core, Momentum Pro, Inscription, R-design, T3, MY22'],
       ['Volvo XC40 Momentum Core, Momentum Pro, Inscription, R-design, T3, Manual, MY22']],
      dtype=object)

In [17]:
type(df_coches[(df_coches['Modelo'] == input_Coche)]["Consumo medio l/1km"].values[0])

numpy.float64

In [22]:
consumo = consumoEleccion(input_Coche,df_coches)
consumo

0.07300000000000001

In [45]:
resultado,prediccion = predicePrecio(input_Coche,df_coches,consumo,input_usuarioKm)

1/1 [==============================] - 0s 368ms/step


In [46]:
prediccion,resultado

(1.6165823, 4.13036771118641)

In [40]:
resultado

4.13036771118641

### Funcion para simular el calculo del precio del combustible de los vehiculos

In [47]:
#input_usuarioPrecio = 1.66 viene incluida en la funcion de prediccion
#Preguntar a DAni si llamar a una funcion por cada un tipo de motorizacion o individual por cada uno
input_usuarioKm = 35
input_Coche = 'Abarth 595 ABARTH 595 1.4 16v T-JET 107 KW (145 CV) E6D'
#quitar df coches por que esta como variable global
def consumoEleccion(input_Coche,df_coches):
    return df_coches[(df_coches['Modelo'] == input_Coche)]["Consumo medio l/1km"].values[0]
    
def predicePrecio(input_Coche,df_coches,consumo,km):
    if ((df_coches[df_coches['Modelo'] == input_Coche]['Motorización']).values[0]  == "Gasolina"  ):
        predic = predicion_gasolina()
        resultado = (predic*consumo)*km

        return resultado,predic
        #LLamar a prediccion gasolina
        pass
    
    elif ((df_coches[df_coches['Modelo'] == input_Coche]['Motorización']).values[0]  == "Gasóleo" ):
        #Prediccion Gasoil
        predic = predicion_gasoleo()
        resultado = (predic*consumo)*km

        return resultado,predic
        pass
    elif ((df_coches[df_coches['Modelo'] == input_Coche]['Motorización']).values[0]  == "Eléctricos puros" ):
        #Prediccion electrica
        pass
    else:
        #llamar a las 2 tipos que use 
        pass


   





In [38]:
def predicion_gasolina():
    T = 10
    df = pd.read_csv(r"D:\bootcamp\Proyecto3\proyecto3sucio\Precios_Gasolina_y_Precios_Gasoleo.csv")
    precio_gasolina = float(input("Ingrese el precio de la gasolina esta semana:"))
    instancia = df["Precio Gasolina"][-10: -1].to_list()
    instancia.append(precio_gasolina)
    instancia = np.array(instancia).reshape(-1, T, 1)

    with open('Data/model_gasolina_pkl.pkl', 'rb') as file:
        modelo_pkl = pickle.load(file)

    prediccion_precio = modelo_pkl.predict(instancia)
    prediccion_precio = prediccion_precio[0][0]
    return(prediccion_precio)

def predicion_gasoleo():
    T = 10
    df = pd.read_csv(r"D:\bootcamp\Proyecto3\proyecto3sucio\Precios_Gasolina_y_Precios_Gasoleo.csv")
    precio_gasoleo = float(input("Ingrese el precio de la gasoleo esta semana:"))
    instancia = df["Precio Gasoleo"][-10: -1].to_list()
    instancia.append(precio_gasoleo)
    instancia = np.array(instancia).reshape(-1, T, 1)
    with open('Data/model_gasoleo_pkl.pkl', 'rb') as file:
        modelo_pkl = pickle.load(file)
    prediccion_precio = modelo_pkl.predict(instancia)
    prediccion_precio = prediccion_precio[0][0]
    return(prediccion_precio)

### Compartiva de los coches segun su tipo

In [53]:
df_coches.Motorización.unique()

array(['Eléctricos puros', 'Gasolina', 'Gasóleo', 'Híbridos enchufables',
       'Híbridos de gasolina', 'Gas natural', 'Híbridos de gasóleo',
       'Gases licuados del petróleo (GLP)'], dtype=object)

In [59]:
df_coches.columns

Index(['Modelo', 'Consumo Mínimo (l/100km)', 'Consumo Máximo (l/100km)',
       'Emisiones Mínimo', 'Emisiones Máximo', 'MTMA (Kg)', 'Motorización',
       'Consumo Eléctrico (kWh/100km)', 'Potencia eléctrica (kW)',
       'Autonomía eléctrica (km)', 'Capacidad de la batería (kWh)',
       'Categoría', 'Consumo Mínimo (kg/100km)', 'Consumo Máximo (kg/100km)',
       'Nº de Plazas', 'L3e', 'L5e', 'L6e', 'L7e', 'M1', 'M2', 'M3', 'N1',
       'N2', 'N3', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'Sin clasificacion',
       'Consumo medio l/1km'],
      dtype='object')

In [89]:
df_coches[df_coches['Motorización'] == 'Gasolina']

,Modelo,Consumo Mínimo (l/100km),Consumo Máximo (l/100km),Emisiones Mínimo,Emisiones Máximo,MTMA (Kg),Motorización,Consumo Eléctrico (kWh/100km),Potencia eléctrica (kW),Autonomía eléctrica (km),...,N2,N3,A,B,C,D,E,F,G,Sin clasificacion
23,Abarth 595 ABARTH 595 1.4 16v T-JET 107 KW (14...,7.2,7.4,162.0,165.0,1045.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
24,Abarth 595 ABARTH 595 COMPETIZIONE 1.4 16v T-J...,7.7,7.8,171.0,172.0,1065.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
25,Abarth 595 ABARTH 595 ESSEESSE 1.4 16v T-JET 1...,7.7,7.8,171.0,172.0,1065.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
26,Abarth 595 ABARTH 595 MONSTER ENERGY YAMAHA 1....,7.4,7.5,165.0,168.0,1065.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
27,Abarth 595 ABARTH 595 PISTA 1.4 16v T-JET 121 ...,7.4,7.5,165.0,168.0,1065.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13200,"Volvo XC40 Gasolina, Essential, Core, Plus, Au...",7.2,7.9,162.0,179.0,2120.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
13206,"Volvo XC40 Momentum Core, Momentum Pro, Inscri...",7.2,7.9,162.0,179.0,2120.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
13207,"Volvo XC40 Momentum Core, Momentum Pro, Inscri...",6.8,7.6,154.0,173.0,2080.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
13208,"Volvo XC40 Momentum Core, Momentum Pro, Inscri...",7.2,7.9,162.0,179.0,2120.0,Gasolina,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [54]:
df_coches[df_coches['Motorización'] == 'Híbridos enchufables'].values[0][0] 

'Alfa Romeo Tonale 1,3 GAS PHEV 280 CV SPECIALE Q4'

In [83]:
#EN combustion entran los de gasolina gasoleo gas natural e gases licuados del petroleo
seleccionCombustion = 'Abarth 595 ABARTH 595 1.4 16v T-JET 107 KW (145 CV) E6D'
#En electricos entran los electricos puros
seleccionElectrico =  'AIWAYS U5 MAS861-WVTA/2WB/FL4'
#En los hibridos entran hibridos de gasoleo y de gasolina
seleccionHibrido =  'Alfa Romeo Tonale 1,3 GAS PHEV 280 CV SPECIALE Q4'

In [72]:
# Supongamos que tienes un DataFrame llamado df_coches con información similar a la proporcionada

# Filtrar los datos para los modelos específicos
modelos_seleccionados = [
    'Abarth 595 ABARTH 595 1.4 16v T-JET 107 KW (145 CV) E6D',
    'AIWAYS U5 MAS861-WVTA/2WB/FL4',
    'Alfa Romeo Tonale 1,3 GAS PHEV 280 CV SPECIALE Q4'
]

df_seleccionados = df_coches[df_coches['Modelo'].isin(modelos_seleccionados)]

# Calcular el consumo promedio para vehículos de combustión
df_seleccionados['Consumo Combustion (l/100km)'] = df_seleccionados[['Consumo Mínimo (l/100km)', 'Consumo Máximo (l/100km)']].mean(axis=1)

# Reorganizar el DataFrame para el gráfico
df_melted = pd.melt(df_seleccionados, id_vars='Modelo', value_vars=['Consumo Eléctrico (kWh/100km)', 'Consumo Combustion (l/100km)'], var_name='Tipo', value_name='Consumo')

# Crear el gráfico de barras con Plotly Express
fig = px.bar(df_melted, x='Modelo', y='Consumo', color='Tipo',
             labels={'Consumo': 'Consumo (unidad por 100 km)', 'Tipo': 'Tipo de Consumo'},
             title='Comparación de Consumo entre Vehículos',
             facet_col='Tipo', facet_col_wrap=1,
             barmode='group')

# Ajustar diseño para mejorar la visualización
fig.update_layout(height=600, width=800)

# Mostrar el gráfico
fig.show()

C:\Users\Danie\AppData\Local\Temp\ipykernel_6796\2093733745.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [91]:
# Crear el DataFrame para cada tipo de vehículo
df_electrico = df_coches[df_coches['Modelo'] == seleccionElectrico]
df_combustion = df_coches[df_coches['Modelo'] == seleccionCombustion]
df_hibrido = df_coches[df_coches['Modelo'] == seleccionHibrido]


# Calcular el consumo medio para el híbrido (promedio entre Consumo Mínimo y Máximo)
df_hibrido['Consumo Medio (l/100km)'] = (df_hibrido['Consumo Mínimo (l/100km)'] + df_hibrido['Consumo Máximo (l/100km)']) / 2

# Crear el gráfico para vehículos eléctricos
fig_electrico = px.bar(df_electrico, x='Modelo', y='Consumo Eléctrico (kWh/100km)',
                       title='Consumo Eléctrico para Vehículos Eléctricos Puros')

# Crear el gráfico para vehículos de combustión
fig_combustion = px.bar(df_combustion, x='Modelo', y=['Consumo Mínimo (l/100km)', 'Consumo Máximo (l/100km)'],
                        labels={'value': 'Consumo (unidad por 100 km)', 'variable': 'Tipo de Consumo'},
                        title=f'Consumo para Combustion', barmode='group')

# Crear el gráfico para vehículos híbridos
fig_hibrido = px.bar(df_hibrido, x='Modelo', y=['Consumo Eléctrico (kWh/100km)', 'Consumo Medio (l/100km)'],
                     labels={'value': 'Consumo (unidad por 100 km)', 'variable': 'Tipo de Consumo'},
                     title=f'Consumo para Hibridos', barmode='group')

# Mostrar los gráficos
fig_electrico.show()
fig_combustion.show()
fig_hibrido.show()


C:\Users\Danie\AppData\Local\Temp\ipykernel_6796\2956823531.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Caracteristicas eléctricas de los vehiculos 

In [207]:
df_coches = df[df['Motorización'].isin(['Eléctricos puros', 'Híbridos enchufables'])]

In [208]:
df_coches.shape

(3659, 33)

## Mostrar la autonomía eléctrica de los vehículos.

In [209]:
def autonomiaElectricos(df_coches):
    """"
    Se devuelve 2 graficos de la autonomia la primera de electricos puros y la segunda de electricos enchufables

    """
    # Filtrar el DataFrame para obtener solo eléctricos puros y híbridos enchufables
    df_filt = df_coches[df_coches['Motorización'].isin(['Eléctricos puros', 'Híbridos enchufables'])]

    # Crear figuras de subgráficos con alturas ajustadas
    fig = sp.make_subplots(rows=2, cols=1, subplot_titles=['Eléctricos puros', 'Híbridos enchufables'],
                        row_heights=[0.6, 0.6])  # Ajustar las alturas aquí

    # Agregar cajas de box plot para Autonomía Eléctrica
    fig.add_trace(
        go.Box(y=df_filt[df_filt['Motorización']=='Eléctricos puros']['Autonomía eléctrica (km)'],
            name='Eléctricos puros',
            marker=dict(color='tomato')),
        row=1, col=1
    )

    fig.add_trace(
        go.Box(y=df_filt[df_filt['Motorización']=='Híbridos enchufables']['Autonomía eléctrica (km)'],
            name='Híbridos enchufables',
            marker=dict(color='lightblue')),
        row=2, col=1
    )

    # Actualizar el diseño de los subgráficos
    fig.update_layout(
        title='Distribución de Autonomía Eléctrica',
        boxmode='group',  # Agrupar las cajas
        xaxis_title='Motorización',
        yaxis_title='Autonomía eléctrica (km)',
        showlegend=False,  # No mostrar leyenda en este caso
        height=1000  # Ajustar la altura manualmente
    )

    fig.show()
print(help(autonomiaElectricos))
autonomiaElectricos(df_coches)

Help on function autonomiaElectricos in module __main__:

autonomiaElectricos(df_coches)
    "
    Se devuelve 2 graficos de la autonomia la primera de electricos puros y la segunda de electricos enchufables

None


In [210]:
def autonomiaConjunta(df_coches):
    """
        Devuelve la grafica de la autonomia conjunta entre electricos puros y enchufables
    """
    fig = px.box(df_coches, y = 'Autonomía eléctrica (km)', title = 'Distribución de Autonomía Eléctrica',
                labels = {'Autonomía eléctrica (km)': 'Autonomía eléctrica (km)'},
                color_discrete_sequence = ['tomato' ])

    fig.show()
print(help(autonomiaConjunta))
autonomiaConjunta(df_coches)

Help on function autonomiaConjunta in module __main__:

autonomiaConjunta(df_coches)
    Devuelve la grafica de la autonomia conjunta entre electricos puros y enchufables

None


In [211]:
menorA133 = df_coches[df_coches['Autonomía eléctrica (km)'] <= 133].shape[0]
entre133Y285 = df_coches[(df_coches['Autonomía eléctrica (km)'] > 133) &  (df_coches['Autonomía eléctrica (km)'] <= 285) ].shape[0]
mayor285 = df_coches[df_coches['Autonomía eléctrica (km)'] > 285].shape[0]

In [212]:
print(f'Numero de vehiculos con autonomia menor a 133 :  {menorA133}')
print(f'Numero de vehiculos con autonomia mayor a 133 y menor de 285 :  {entre133Y285}')
print(f'Numero de vehiculos con autonomia mayor a 285 :  {mayor285}')


Numero de vehiculos con autonomia menor a 133 :  1839
Numero de vehiculos con autonomia mayor a 133 y menor de 285 :  906
Numero de vehiculos con autonomia mayor a 285 :  878


Cambiar esto
En la grafica se aprecia que la autonomia de los vehiculos esta condensada entre 106 y 323 km desglosado de la siguiente manera:

-1839 tienen un valor de autonomia del vehiculo menor de 133 por lo cual hay una tendencia 
alta en la fabricacion/venta de este tipo de vehiculos.

-906 vehiculos con autonomia mayor a 133 y menor que 285 y 710 mayor a 285 km de autonomia sugiere que
la tendencia en este punto de fabricacion/venta esta repartida casi equanime entre las dos.

Conclusion:
Se entiende que los vehiculos electricos mas populares son los que tienen una autonomia menor a 133 km.






In [213]:
def violinAutonomiaConjunta(df_coches):
    """
    Se devuelve la grafica de violin de la autonomia de coches electricos puros e hibridos
    """
    fig = px.violin(df_coches, y='Autonomía eléctrica (km)', box=True, title='Distribución de Autonomía Eléctrica',
                    labels={'Autonomía eléctrica (km)': 'Autonomía eléctrica (km)'},
                    color_discrete_sequence= ['tomato'])

    fig.show()
print(help(violinAutonomiaConjunta))
violinAutonomiaConjunta(df_coches)


Help on function violinAutonomiaConjunta in module __main__:

violinAutonomiaConjunta(df_coches)
    Se devuelve la grafica de violin de la autonomia de coches electricos puros e hibridos

None


Como en la anterior grafica si observamos el KDE denota que la maxima intensidad esta entre los menores de 185 afirmando lo que nos sugeria
la anterior grafica

In [221]:
df_cochesModa = df_coches[(df_coches['Autonomía eléctrica (km)'] <= 133) & (df_coches['Autonomía eléctrica (km)'] > 10.00)]

Dentro de la moda es decir la mayor densidad de autonomias electricas de los vehiculos ¿que tipo de vehiculos hay?

In [222]:
def autonomiasPorCategoriaElectricosPuros(df_cochesModa):
    """
    Devuelve la grafica que relaciona la autonomi de los vehiculos electricos puros con su categoria
    """
   

    # Filtra para obtener solo vehículos eléctricos
    df_electricos = df_cochesModa[df_cochesModa['Motorización'] == 'Eléctricos puros']

    # Selecciona solo las columnas de categorías dummy y la autonomía eléctrica
    columnas_categorias = ['L3e', 'L5e', 'L6e', 'L7e', 'M1', 'M2', 'M3', 'N1', 'N2', 'N3']
    df_categorias = df_electricos[['Autonomía eléctrica (km)'] + columnas_categorias]

    # Reshape del DataFrame para facilitar la visualización
    df_categorias_melted = pd.melt(df_categorias, id_vars='Autonomía eléctrica (km)', var_name='Categoría', value_name='Pertenece')

    # Filtra solo los casos donde Pertenece es 1
    df_categorias_melted = df_categorias_melted[df_categorias_melted['Pertenece'] == 1]

    # Crea un gráfico historiplot
    fig = px.histogram(df_categorias_melted, x='Autonomía eléctrica (km)', color='Categoría',
                    marginal='rug',  # Añade "rug plot" en los márgenes
                    title='Historiplot de Autonomía Eléctrica por Categoría En electricos puros',
                    labels={'Autonomía eléctrica (km)': 'Autonomía eléctrica (km)'},
                    color_discrete_sequence=px.colors.qualitative.Set3)  # Paleta de colores

    # Muestra el gráfico
    fig.show()
print(help(autonomiasPorCategoriaElectricosPuros))
autonomiasPorCategoriaElectricosPuros(df_cochesModa)

Help on function autonomiasPorCategoriaElectricosPuros in module __main__:

autonomiasPorCategoriaElectricosPuros(df_cochesModa)
    Devuelve la grafica que relaciona la autonomi de los vehiculos electricos puros con su categoria

None


In [224]:

def autonomiasPorCategoriaHibridosEnchufables(df_cochesModa):
    """
    Devuelve la grafica que relaciona la autonomi de los vehiculos hibridos enchufables con su categoria
    """

    # Filtra para obtener solo vehículos eléctricos
    df_electricos = df_cochesModa[df_cochesModa['Motorización'] == 'Híbridos enchufables']

    # Selecciona solo las columnas de categorías dummy y la autonomía eléctrica
    columnas_categorias = ['L3e', 'L5e', 'L6e', 'L7e', 'M1', 'M2', 'M3', 'N1', 'N2', 'N3']
    df_categorias = df_electricos[['Autonomía eléctrica (km)'] + columnas_categorias]

    # Reshape del DataFrame para facilitar la visualización
    df_categorias_melted = pd.melt(df_categorias, id_vars='Autonomía eléctrica (km)', var_name='Categoría', value_name='Pertenece')

    # Filtra solo los casos donde Pertenece es 1
    df_categorias_melted = df_categorias_melted[df_categorias_melted['Pertenece'] == 1]

    # Crea un gráfico historiplot
    fig = px.histogram(df_categorias_melted, x='Autonomía eléctrica (km)', color='Categoría',
                    marginal='rug',  # Añade "rug plot" en los márgenes
                    title='Historiplot de Autonomía Eléctrica por Categoría en hibridos enchufables',
                    labels={'Autonomía eléctrica (km)': 'Autonomía eléctrica (km)'},
                    color_discrete_sequence=px.colors.qualitative.Set3)  # Paleta de colores

    # Muestra el gráfico
    fig.show()
print(help(autonomiasPorCategoriaHibridosEnchufables))
autonomiasPorCategoriaHibridosEnchufables(df_cochesModa)

Help on function autonomiasPorCategoriaHibridosEnchufables in module __main__:

autonomiasPorCategoriaHibridosEnchufables(df_cochesModa)
    Devuelve la grafica que relaciona la autonomi de los vehiculos hibridos enchufables con su categoria

None


In [225]:
def autonomiasPorCategoriaConjunto(df_cochesModa):
    """
    Se devuelve el grafico que relaciona la categoria y la autonomia de todos los vehiculos electricos(hibridos y puros)
    """
    # Filtra para obtener solo vehículos eléctricos
    df_electricos = df_cochesModa.copy()

    # Selecciona solo las columnas de categorías dummy y la autonomía eléctrica
    columnas_categorias = ['L3e', 'L5e', 'L6e', 'L7e', 'M1', 'M2', 'M3', 'N1', 'N2', 'N3']
    df_categorias = df_electricos[['Autonomía eléctrica (km)'] + columnas_categorias]

    # Reshape del DataFrame para facilitar la visualización
    df_categorias_melted = pd.melt(df_categorias, id_vars='Autonomía eléctrica (km)', var_name='Categoría', value_name='Pertenece')

    # Filtra solo los casos donde Pertenece es 1
    df_categorias_melted = df_categorias_melted[df_categorias_melted['Pertenece'] == 1]

    # Crea un gráfico historiplot
    fig = px.histogram(df_categorias_melted, x='Autonomía eléctrica (km)', color='Categoría',
                    marginal='rug',  # Añade "rug plot" en los márgenes
                    title='Historiplot de Autonomía Eléctrica por Categoría En electricos puros',
                    labels={'Autonomía eléctrica (km)': 'Autonomía eléctrica (km)'},
                    color_discrete_sequence=px.colors.qualitative.Set3)  # Paleta de colores

    # Muestra el gráfico
    fig.show()
print(help(autonomiasPorCategoriaConjunto))
autonomiasPorCategoriaConjunto(df_cochesModa)

Help on function autonomiasPorCategoriaConjunto in module __main__:

autonomiasPorCategoriaConjunto(df_cochesModa)
    Se devuelve el grafico que relaciona la categoria y la autonomia de todos los vehiculos electricos(hibridos y puros)

None


In [226]:
df_cochesModa['Autonomía eléctrica (km)'].sort_values()

6395      13.0
6396      13.0
6136      13.0
1771      31.0
16288     33.0
         ...  
5371     133.0
5372     133.0
5373     133.0
12373    133.0
5363     133.0
Name: Autonomía eléctrica (km), Length: 1833, dtype: float64

In [227]:
#codigo para sacar un diccionario con las categorias y los max media y min de cada una
# Lista de categorías
categorias = ['L3e', 'L5e', 'L6e', 'L7e', 'M1', 'M2', 'M3', 'N1', 'N2', 'N3']

# Crear un diccionario para almacenar los resultados
resultados_por_categoria = {}

# Iterar sobre cada categoría
for categoria in categorias:
    # Filtrar el DataFrame por la categoría actual
    df_categoria = df_cochesModa[df_cochesModa[categoria] == 1]
    
    # Obtener las estadísticas descriptivas para la Autonomía eléctrica
    estadisticas_descriptivas = df_categoria['Autonomía eléctrica (km)'].describe()
    
    # Almacenar las estadísticas en el diccionario
    resultados_por_categoria[categoria] = estadisticas_descriptivas.to_dict()

# Imprimir el diccionario de resultados
print(resultados_por_categoria)

{'L3e': {'count': 196.0, 'mean': 95.96938775510205, 'std': 18.063394962518863, 'min': 66.0, '25%': 80.0, '50%': 94.5, '75%': 108.25, 'max': 133.0}, 'L5e': {'count': 5.0, 'mean': 104.2, 'std': 6.572670690061994, 'min': 97.0, '25%': 97.0, '50%': 109.0, '75%': 109.0, 'max': 109.0}, 'L6e': {'count': 59.0, 'mean': 89.7457627118644, 'std': 23.44957408652181, 'min': 48.0, '25%': 72.0, '50%': 98.0, '75%': 101.0, 'max': 130.0}, 'L7e': {'count': 275.0, 'mean': 86.07272727272728, 'std': 24.083533570681986, 'min': 45.0, '25%': 72.0, '50%': 83.0, '75%': 96.0, 'max': 133.0}, 'M1': {'count': 763.0, 'mean': 61.09239842726081, 'std': 14.765255730461623, 'min': 13.0, '25%': 51.0, '50%': 60.0, '75%': 66.0, 'max': 125.0}, 'M2': {'count': 0.0, 'mean': nan, 'std': nan, 'min': nan, '25%': nan, '50%': nan, '75%': nan, 'max': nan}, 'M3': {'count': 0.0, 'mean': nan, 'std': nan, 'min': nan, '25%': nan, '50%': nan, '75%': nan, 'max': nan}, 'N1': {'count': 449.0, 'mean': 90.55233853006682, 'std': 19.47107630819179

Dentro de la moda (autonomia menor de 133 km )  se observa que la categoria con mayor numero de vehiculos es la M1
que corresponde a  vehiculos destinados al transporte de pasajeros con un maximo de 8 asientos ademas del asiento del conductor.

A continuacion se muestra el conteo de las categorias dentro de la moda Min Media y Max pertenecen a la autonomia (km): 

| Categoria | Min  | Media | Max  |
|-----------|------|-------|------|
| L3e       | 66.0 | 95.97 | 133.0|
| L5e       | 97.0 | 104.2 | 109.0|
| L6e       | 48.0 | 89.75 | 130.0|
| L7e       | 45.0 | 86.07 | 133.0|
| M1        | 13.0 | 61.09 | 125.0|
| M2        | nan  | nan   | nan  |
| M3        | nan  | nan   | nan  |
| N1        | 41.0 | 90.55 | 130.0|
| N2        | 100.0| 117.28| 120.0|
| N3        | 60.0 | 76.25 | 110.0|






